In [1]:
import json
import pprint

In [2]:
import z3
#https://z3prover.github.io/api/html/z3py_8py_source.html

In [3]:
optimizer = z3.Optimize()
optimizer.help()

dump_benchmarks (bool) dump benchmarks for profiling (default: false)
elim_01 (bool) eliminate 01 variables (default: true)
enable_sat (bool) enable the new SAT core for propositional constraints (default: true)
enable_sls (bool) enable SLS tuning during weighted maxsast (default: false)
maxres.add_upper_bound_block (bool) restict upper bound with constraint (default: false)
maxres.hill_climb (bool) give preference for large weight cores (default: true)
maxres.max_core_size (unsigned int) break batch of generated cores if size reaches this number (default: 3)
maxres.max_correction_set_size (unsigned int) allow generating correction set constraints up to maximal size (default: 3)
maxres.max_num_cores (unsigned int) maximal number of cores per round (default: 4294967295)
maxres.maximize_assignment (bool) find an MSS/MCS to improve current assignment (default: false)
maxres.pivot_on_correction_set (bool) reduce soft constraints if the current correction set is smaller than current core (d

In [4]:
optimizer = z3.Optimize()
x=z3.Int('x') # (declare-const x Int)
y=z3.Int('y') #(declare-const y Int)

print(optimizer.param_descrs())
optimizer.assert_exprs(x < 2)#(assert (< x 2))
optimizer.assert_exprs((y-x)>1)#(assert (> (- y x) 1))
my_max = optimizer.maximize(x+y)#(maximize (+ x y))
my_min = optimizer.minimize(x+y)
#optimizer.set("priority", "box")
optimizer.check()#(check-sat)
print(optimizer.lower(my_min))
print(optimizer.lower(my_max))
print(optimizer.upper(my_min))
print(optimizer.upper(my_max))
print(optimizer.model())
#print(optimizer.statistics())

(optsmt_engine, maxsat_engine, priority, dump_benchmarks, print_model, enable_sls, enable_sat, elim_01, pp.neat, pb.compile_equality, maxres.hill_climb, maxres.add_upper_bound_block, maxres.max_num_cores, maxres.max_core_size, maxres.maximize_assignment, maxres.max_correction_set_size, maxres.wmax, maxres.pivot_on_correction_set)
-1*oo
oo
oo
oo
[y = 5, x = 1]


In [5]:
z3.set_param(verbose=10)
optimizer = z3.Optimize()
x=z3.Int('x')#(declare-const x Int)
y=z3.Int('y')#(declare-const y Int)
optimizer.assert_exprs(x < 4)#(assert (< x 4))
optimizer.assert_exprs((y-x) < 1)#(assert (< (- y x) 1))
optimizer.assert_exprs(y < 1)#(assert (< y 1))
optimizer.assert_exprs(y>-10)

In [6]:
my_min=optimizer.minimize(x+y)#(minimize (+ x y))

In [7]:
optimizer.check()#(check-sat)

sat

In [8]:
optimizer.model()

[y = -9, x = -9]

In [9]:
optimizer.lower(my_min)

-18

In [10]:
optimizer.upper(my_min)

-18

In [11]:
with open ('dummy_km.json') as file:
    data = json.load(file)
    
#pprint.pprint(data)

In [12]:
data['VG1']['dependencies']['nonlocal']

['APP2', 'APP3']

I think there needs to be a list of all resource types. I'm not sure how to guarantee the ordering otherwise. 

In [13]:
[j for i in data for j in data[i]['resources'] ]#depth first, pull all elements 

['storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory']

In [14]:
resourceTypes = set([j for i in data for j in data[i]['resources'] ])
print(resourceTypes)

{'storage', 'memory'}


nodeList

In [15]:
nodeList = list()
for i in data:
    if data[i]['type']=='node':
        nodeList.append(i)
print (nodeList)

['FSSN2', 'FSSN3', 'FSSN1']


In [16]:
appList = list()
for i in data:
    if data[i]['type']=='app':
        appList.append(i)
print (appList)

['APP2', 'VG1', 'APP3']


In [17]:
a2n = [ [ z3.Int("a2n_%s_%s" % (i, j)) for j in range(len(nodeList)) ]
            for i in range(len(appList)) ]
print(a2n)

[[a2n_0_0, a2n_0_1, a2n_0_2], [a2n_1_0, a2n_1_1, a2n_1_2], [a2n_2_0, a2n_2_1, a2n_2_2]]


In [18]:
a2n = [ [ z3.Int("a2n_%son%s" % (i, j)) for j in nodeList ]
            for i in appList]
pprint.pprint(a2n)
print(a2n[0][2])

[[a2n_APP2onFSSN2, a2n_APP2onFSSN3, a2n_APP2onFSSN1],
 [a2n_VG1onFSSN2, a2n_VG1onFSSN3, a2n_VG1onFSSN1],
 [a2n_APP3onFSSN2, a2n_APP3onFSSN3, a2n_APP3onFSSN1]]
a2n_APP2onFSSN1


matrix for resource x node matrix

In [19]:
#build and check format for matrix of node resources
rsrc_per_node = [[z3.Int("rpn_%s_%s" %(k, j)) for j in nodeList] for k in resourceTypes]
rpn= [[data[j]['resources'][k] for j in nodeList] for k in resourceTypes]

k = 1
j = 0
pprint.pprint (rsrc_per_node)
pprint.pprint(rpn)
print("index [{}][{}] {} {}".format(k,j, rsrc_per_node[k][j], rpn[k][j]))

[[rpn_storage_FSSN2, rpn_storage_FSSN3, rpn_storage_FSSN1],
 [rpn_memory_FSSN2, rpn_memory_FSSN3, rpn_memory_FSSN1]]
[[512, 1024, 1024], [256, 512, 1024]]
index [1][0] rpn_memory_FSSN2 256


In [20]:
#build and check format for matrix of app resource requirements
print ([i for i in appList])
rsrc_per_app = [[z3.Int("rpa_%s_%s" %(k, i)) for i in appList] for k in resourceTypes]

k = 1
i = 1

pprint.pprint(rsrc_per_app)
rpa= [[data[i]['resources'][k] for i in appList] for k in resourceTypes]
print("index [{}][{}] {} {}".format(k,i, rsrc_per_app[k][i], rpa[k][i]))

['APP2', 'VG1', 'APP3']
[[rpa_storage_APP2, rpa_storage_VG1, rpa_storage_APP3],
 [rpa_memory_APP2, rpa_memory_VG1, rpa_memory_APP3]]
index [1][1] rpa_memory_VG1 128


In [21]:
# two ways to build latency matrix. The first way does not guarantee the order of the first index when
#reading from the json, but does the second. The second way gurantees nothing. 
# We could do a third way that ensures the FSSNs are in order too. 
#latency = []
#for node in nodeList:
#    print(node)
#    id = data[node]['id']
#    temp = []
#    for index in range(len(data[node]['latencies'])):
        #print (index)
#        lat = data[node]['latencies']['n%s%s' %(id, index+1)]
#        print (lat)
#        temp.append(lat)
#    latency.append(temp)
#    print (latency)
    
latency = []
for node in nodeList:
    print(node)
    id = data[node]['id']
    temp = []
    for target in data[node]['latencies']:
        lat = data[node]['latencies'][target]
        print ("%s : %s" %(target, lat))
        temp.append(lat)
    latency.append(temp)
print (latency)

FSSN2
n22 : 0
n21 : 1
n23 : 1
FSSN3
n33 : 0
n32 : 1
n31 : 1
FSSN1
n12 : 1
n11 : 0
n13 : 1
[[0, 1, 1], [0, 1, 1], [1, 0, 1]]


In [22]:
#make sure resources match when k changes. 

k = 1
i = 0
j = 1
print("index [{}][{}] {} {}".format(k,j, rsrc_per_node[k][j], rpn[k][j]))
print("index [{}][{}] {} {}".format(k,i, rsrc_per_app[k][i], rpa[k][i]))

rpn[k][j] >= rpa[k][i]

index [1][1] rpn_memory_FSSN3 512
index [1][0] rpa_memory_APP2 128


True

In [23]:
val_a2n = [ z3.Or(a2n[i][j]==0,a2n[i][j]==1) for j in range(len(nodeList))
          for i in range(len(appList)) ]
pprint.pprint(val_a2n)

[Or(a2n_APP2onFSSN2 == 0, a2n_APP2onFSSN2 == 1),
 Or(a2n_VG1onFSSN2 == 0, a2n_VG1onFSSN2 == 1),
 Or(a2n_APP3onFSSN2 == 0, a2n_APP3onFSSN2 == 1),
 Or(a2n_APP2onFSSN3 == 0, a2n_APP2onFSSN3 == 1),
 Or(a2n_VG1onFSSN3 == 0, a2n_VG1onFSSN3 == 1),
 Or(a2n_APP3onFSSN3 == 0, a2n_APP3onFSSN3 == 1),
 Or(a2n_APP2onFSSN1 == 0, a2n_APP2onFSSN1 == 1),
 Or(a2n_VG1onFSSN1 == 0, a2n_VG1onFSSN1 == 1),
 Or(a2n_APP3onFSSN1 == 0, a2n_APP3onFSSN1 == 1)]


In [24]:
rsrc_constraint = [rpn[k][j] >= z3.Sum([a2n[i][j]*rpa[k][i] for i in range(len(appList))]) 
 for j in range(len(nodeList))
 for k in range(len(resourceTypes))]

In [25]:
md_constraint1 = [z3.Sum(a2n[0][0], a2n[0][1], a2n[0][2])==1]
md_constraint2 = [z3.Sum(a2n[1][0], a2n[1][1], a2n[1][2])==1]


vg1 = appList.index('VG1')
print(vg1)
md_all = [z3.And([a2n[vg1][node]==1 for node in range(len(nodeList))])]
print(md_all)

1
[And(a2n_VG1onFSSN2 == 1,
    a2n_VG1onFSSN3 == 1,
    a2n_VG1onFSSN1 == 1)]


In [26]:
#dependency matrix c2c
a2a = []
for app in appList:
    print(app)
    nl_deps = data[app]['dependencies']['nonlocal']
    deps = []
    for dep in appList:
        if dep in nl_deps:
            print("depends on %s" %(dep))
            deps.append(1)
        else: 
            print("does not depend on %s" %(dep))
            deps.append(0)
            
    a2a.append(deps)
print ("a2a matrix %s" %a2a)
print (a2a[appList.index('VG1')][appList.index('VG1')])
print (a2a[appList.index('VG1')][appList.index('APP2')])
print (a2a[appList.index('VG1')][appList.index('APP3')])
print(a2a[appList.index('VG1')])
#latency = []
#for node in nodeList:
#    print(node)
#    id = data[node]['id']
#    temp = []
#    for target in data[node]['latencies']:
#        lat = data[node]['latencies'][target]
#        print ("%s : %s" %(target, lat))
#        temp.append(lat)
#    latency.append(temp)
#print (latency)

APP2
does not depend on APP2
does not depend on VG1
does not depend on APP3
VG1
depends on APP2
does not depend on VG1
depends on APP3
APP3
does not depend on APP2
does not depend on VG1
does not depend on APP3
a2a matrix [[0, 0, 0], [1, 0, 1], [0, 0, 0]]
0
1
1
[1, 0, 1]


In [27]:
#dependency constraints
dependencies = list()

def DependsOn(pack, deps):
    print("deps are %s" %deps)
    return z3.And([ z3.Implies(pack, dep) for dep in deps ])

print ("appList: %s" % appList)
for app in appList:
    appi = appList.index(app)
    nl_deps = data[app]['dependencies']['nonlocal']
    print("%s non-local dependencies: %s" %(app, nl_deps))
    if nl_deps:
        #depi = [appList.index(dep) for dep in nl_deps]
        #print ("dependency index: %s" %depi)
        dependencies = [DependsOn(a2n[appi][nodej]==1,[a2n[appList.index(dep)][nodej]==1 for dep in nl_deps]) for nodej in range(len(nodeList))]
        #print (dependencies)
        test = [[a2n[appList.index(dep)][nodej]==1 for dep in nl_deps] for nodej in range(len(nodeList))]
        #print (test)
        #pre = [z3.Sum([a2n[appi][nodej] for nodej in range(len(nodeList))]) > 0]
        pre = z3.Or([a2n[appi][nodej]==1 for nodej in range(len(nodeList))])
        print("pre %s" %pre)
        post = list()
        for dep in nl_deps:            
            post.append(z3.Or([a2n[appList.index(dep)][nodej]==1 for nodej in range(len(nodeList))]))
            
            
        print("post %s" %post)
        dependencies = [DependsOn(pre, post)]
        print (dependencies)
        #z3.Or(z3.Not(pre), post)
#a2n[appi][nodej]==1, z3.Or()
#[ z3.Or(a2n[i][j]==0,a2n[i][j]==1) for j in range(len(nodeList)) for i in range(len(appList)) ]

    

appList: ['APP2', 'VG1', 'APP3']
APP2 non-local dependencies: []
VG1 non-local dependencies: ['APP2', 'APP3']
deps are [a2n_APP2onFSSN2 == 1, a2n_APP3onFSSN2 == 1]
deps are [a2n_APP2onFSSN3 == 1, a2n_APP3onFSSN3 == 1]
deps are [a2n_APP2onFSSN1 == 1, a2n_APP3onFSSN1 == 1]
pre Or(a2n_VG1onFSSN2 == 1,
   a2n_VG1onFSSN3 == 1,
   a2n_VG1onFSSN1 == 1)
post [Or(a2n_APP2onFSSN2 == 1,
   a2n_APP2onFSSN3 == 1,
   a2n_APP2onFSSN1 == 1), Or(a2n_APP3onFSSN2 == 1,
   a2n_APP3onFSSN3 == 1,
   a2n_APP3onFSSN1 == 1)]
deps are [Or(a2n_APP2onFSSN2 == 1,
   a2n_APP2onFSSN3 == 1,
   a2n_APP2onFSSN1 == 1), Or(a2n_APP3onFSSN2 == 1,
   a2n_APP3onFSSN3 == 1,
   a2n_APP3onFSSN1 == 1)]
[And(Implies(Or(a2n_VG1onFSSN2 == 1,
               a2n_VG1onFSSN3 == 1,
               a2n_VG1onFSSN1 == 1),
            Or(a2n_APP2onFSSN2 == 1,
               a2n_APP2onFSSN3 == 1,
               a2n_APP2onFSSN1 == 1)),
    Implies(Or(a2n_VG1onFSSN2 == 1,
               a2n_VG1onFSSN3 == 1,
               a2n_VG1onFSSN1 == 1),


In [28]:
solver = z3.Solver()
solver.add(val_a2n + rsrc_constraint + md_all+dependencies )
pprint.pprint(val_a2n + rsrc_constraint + md_all+dependencies)

[Or(a2n_APP2onFSSN2 == 0, a2n_APP2onFSSN2 == 1),
 Or(a2n_VG1onFSSN2 == 0, a2n_VG1onFSSN2 == 1),
 Or(a2n_APP3onFSSN2 == 0, a2n_APP3onFSSN2 == 1),
 Or(a2n_APP2onFSSN3 == 0, a2n_APP2onFSSN3 == 1),
 Or(a2n_VG1onFSSN3 == 0, a2n_VG1onFSSN3 == 1),
 Or(a2n_APP3onFSSN3 == 0, a2n_APP3onFSSN3 == 1),
 Or(a2n_APP2onFSSN1 == 0, a2n_APP2onFSSN1 == 1),
 Or(a2n_VG1onFSSN1 == 0, a2n_VG1onFSSN1 == 1),
 Or(a2n_APP3onFSSN1 == 0, a2n_APP3onFSSN1 == 1),
 a2n_APP2onFSSN2*256 +
a2n_VG1onFSSN2*256 +
a2n_APP3onFSSN2*256 <=
512,
 a2n_APP2onFSSN2*128 +
a2n_VG1onFSSN2*128 +
a2n_APP3onFSSN2*128 <=
256,
 a2n_APP2onFSSN3*256 +
a2n_VG1onFSSN3*256 +
a2n_APP3onFSSN3*256 <=
1024,
 a2n_APP2onFSSN3*128 +
a2n_VG1onFSSN3*128 +
a2n_APP3onFSSN3*128 <=
512,
 a2n_APP2onFSSN1*256 +
a2n_VG1onFSSN1*256 +
a2n_APP3onFSSN1*256 <=
1024,
 a2n_APP2onFSSN1*128 +
a2n_VG1onFSSN1*128 +
a2n_APP3onFSSN1*128 <=
1024,
 And(a2n_VG1onFSSN2 == 1,
    a2n_VG1onFSSN3 == 1,
    a2n_VG1onFSSN1 == 1),
 And(Implies(Or(a2n_VG1onFSSN2 == 1,
               a

In [29]:
solver.check()

sat

In [30]:
solver.model()

[a2n_APP3onFSSN2 = 1,
 a2n_APP3onFSSN3 = 0,
 a2n_APP2onFSSN1 = 1,
 a2n_APP3onFSSN1 = 0,
 a2n_APP2onFSSN3 = 0,
 a2n_APP2onFSSN2 = 0,
 a2n_VG1onFSSN1 = 1,
 a2n_VG1onFSSN3 = 1,
 a2n_VG1onFSSN2 = 1]

In [31]:
for node in nodeList:
    print("latency on node %s: %s" %(node, data[node]['latencies']))

latency on node FSSN2: {'n22': 0, 'n21': 1, 'n23': 1}
latency on node FSSN3: {'n33': 0, 'n32': 1, 'n31': 1}
latency on node FSSN1: {'n12': 1, 'n11': 0, 'n13': 1}


In [32]:
a2n

[[a2n_APP2onFSSN2, a2n_APP2onFSSN3, a2n_APP2onFSSN1],
 [a2n_VG1onFSSN2, a2n_VG1onFSSN3, a2n_VG1onFSSN1],
 [a2n_APP3onFSSN2, a2n_APP3onFSSN3, a2n_APP3onFSSN1]]

In [33]:
#compute latency of a particular placement
lat_s = list()
lat_t = list()
for app in appList:
    nl_deps = data[app]['dependencies']['nonlocal']
    print(nl_deps)
    if nl_deps: # if there are non-local dependencies
        for dep in nl_deps:
            for node in nodeList:
                if a2n[appList.index(app)][nodeList.index(node)]== 1 :
                    lat_s.append(node)
                    print (True)
                else: print(False)
                        
                if a2n[appList.index(dep)][nodeList.index(node)]== 1 :
                    lat_t.append(node)
print (lat_s)
print (lat_t)
                
                

[]
['APP2', 'APP3']
False
False
False
False
False
False
[]
[]
[]


In [34]:
from functools import reduce
optimizer = z3.Optimize()
constraints = val_a2n + rsrc_constraint + dependencies
optimizer.add(constraints)
print (reduce(lambda x, y: x + sum(y), a2n, 0))
#optimizer.maximize(reduce(lambda x, y: x + sum(y), a2n, 0))
#print(optimizer.check())
#print(optimizer.model())

0 +
0 +
a2n_APP2onFSSN2 +
a2n_APP2onFSSN3 +
a2n_APP2onFSSN1 +
0 +
a2n_VG1onFSSN2 +
a2n_VG1onFSSN3 +
a2n_VG1onFSSN1 +
0 +
a2n_APP3onFSSN2 +
a2n_APP3onFSSN3 +
a2n_APP3onFSSN1


In [35]:
import numpy

ImportError: No module named 'numpy'